In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv


load_dotenv(dotenv_path='../.env')

DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME') 

# 1. Leitura dos dados brutos
caminho_csv = '../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df_clientes = pd.read_csv(caminho_csv)
print("Dados originais carregados com sucesso!")

# 2. Limpeza e padronização dos dados

# 2.1 Padronizando os nomes das colunas
novas_colunas = {
    'customerID': 'customer_id',
    'gender': 'gender',
    'SeniorCitizen': 'senior_citizen',
    'Partner': 'partner',
    'Dependents': 'dependents',
    'tenure': 'tenure',
    'PhoneService': 'phone_service',
    'MultipleLines': 'multiple_lines',
    'InternetService': 'internet_service',
    'OnlineSecurity': 'online_security',
    'OnlineBackup': 'online_backup',
    'DeviceProtection': 'device_protection',
    'TechSupport': 'tech_support',
    'StreamingTV': 'streaming_tv',
    'StreamingMovies': 'streaming_movies',
    'Contract': 'contract',
    'PaperlessBilling': 'paperless_billing',
    'PaymentMethod': 'payment_method',
    'MonthlyCharges': 'monthly_charges',
    'TotalCharges': 'total_charges',
    'Churn': 'churn'
}
df_clientes.rename(columns=novas_colunas, inplace=True)

# 2.2 Tratando a coluna 'total_charges'
# Convertendo para numérico e forçando erros (espaços em branco) a virarem 'NaN' (nulos)
df_clientes['total_charges'] = pd.to_numeric(df_clientes['total_charges'], errors='coerce')

# Preenchendo os valores nulos com 0 (já que são clientes novos, o total cobrado é zero)
df_clientes['total_charges'] = df_clientes['total_charges'].fillna(0)

# 3. CARREGAMENTO (ENVIANDO PARA O POSTGRESQL)
print("Iniciando a inserção no banco de dados...")

db_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(db_url)

df_clientes.to_sql('raw_customers', engine, if_exists='append', index=False)

print("SUCESSO! Todos os dados foram inseridos limpos e padronizados no PostgreSQL!")